In [114]:
import json
import os

# 获取数据

In [115]:
def get_1998_news_words():
    words_path = './data/news_1_gram.json'
    with open(words_path, 'r', encoding='utf-8') as f:
        words = json.load(f)
    return words

def get_THUOCL_words():
    data_path = './data/THUOCL'
    # 得到文件列表中的所有文件名
    files = os.listdir(data_path)
    # 遍历文件名列表，将文件中的内容添加到words字典中
    words = {}
    for file in files:
        with open(os.path.join(data_path, file), 'r', encoding='utf-8') as f:
            for line in f:
                line = line.split()
                if len(line) == 2:
                    try:
                        words[line[0]] = int(line[1])
                    except:
                        pass
    return words

def get_n_grams():
    words_path = './data/news_2_gram.json'
    with open(words_path, 'r', encoding='utf-8') as f:
        words = json.load(f)
    return words

def get_words_dic(data_name):
    if data_name == '1998版新闻':
       words = get_1998_news_words()
    elif data_name == 'THUOCL清华数据源':
       words = get_THUOCL_words()
    elif data_name == '词库融合':
        # 统一数据规模
        words = get_1998_news_words()
        THUOCL_words = get_THUOCL_words()
        count_1998_news = sum(list(words.values()))
        count_THUOCL = sum(list(THUOCL_words.values()))
        THUOCL_words = {word: int(count_1998_news * count / count_THUOCL) for word, count in words.items()}
        # 扩充数据集
        words.update(THUOCL_words)
    return words



In [116]:
data_name = 'THUOCL清华数据源'
words_dic = get_words_dic(data_name)
# 展示字典的元素
q = 10 
for word, count in words_dic.items():
    if q > 0:
        print(f'{word}: {count}')
        q -= 1

信鸽: 220963
黄蜂: 118861
水母: 78147
野猫: 62065
宠物狗: 56875
母狗: 56087
毛毛虫: 48631
猎豹: 48451
犀牛: 46120
萨摩: 84


# FMM

In [117]:
def FMM(sentence, words_dic):
    max_length = max(len(word) for word in words_dic)
    sentence = sentence.strip()
    words_length = len(sentence)
    cut_word_list = []

    while words_length > 0:
        max_cut_length = min(max_length, words_length)
        sub_sentence = sentence[0: max_cut_length]
        while max_cut_length > 0: # 最长匹配
            if sub_sentence in words_dic: # 在字典中
                cut_word_list.append(sub_sentence)
                break
            elif max_cut_length == 1: # 不在字典中
                cut_word_list.append(sub_sentence)
                break
            else: # 不在字典中，最大框长度-1
                max_cut_length = max_cut_length - 1
                sub_sentence = sub_sentence[0:max_cut_length]
        sentence = sentence[max_cut_length:]
        words_length = words_length - max_cut_length

    return cut_word_list

# BMM

In [118]:

def BMM(sentence, words_dic):
    max_length = max(len(word) for word in words_dic) # 统计词典中词的最长长度
    sentence = sentence.strip()
    words_length = len(sentence)
    cut_word_list = []

    while words_length > 0:
        max_cut_length = min(max_length, words_length)
        sub_sentence = sentence[-max_cut_length:] # 与FMM不同之处
        while max_cut_length > 0:
            if sub_sentence in words_dic:
                cut_word_list.append(sub_sentence)
                break
            elif max_cut_length == 1:
                cut_word_list.append(sub_sentence)
                break
            else:
                max_cut_length = max_cut_length -1
                sub_sentence = sub_sentence[-max_cut_length:]  # 与FMM不同之处
        sentence = sentence[0:-max_cut_length] # 与FMM不同之处
        words_length = words_length - max_cut_length
    cut_word_list.reverse()

    return cut_word_list

# Bi-MM

In [119]:
# 实现双向匹配算法中的切词方法
def Bi_MM(sentence, words_dic):
    bmm_word_list = BMM(sentence, words_dic)
    fmm_word_list = FMM(sentence, words_dic)
    if bmm_word_list == fmm_word_list:
        return bmm_word_list
    else:
        bmm_word_list_size = len(bmm_word_list)
        fmm_word_list_size = len(fmm_word_list)
        if bmm_word_list_size != fmm_word_list_size:
            return bmm_word_list if bmm_word_list_size < fmm_word_list_size else fmm_word_list
        else:
            FMM_one_word_count = sum([1 for word in fmm_word_list if len(word) == 1])
            BMM_one_word_count = sum([1 for word in bmm_word_list if len(word) == 1])
            return fmm_word_list if BMM_one_word_count > FMM_one_word_count else bmm_word_list

# n-gram算法 

 - 通过最大前向匹配算法和最大后向匹配算法得到两个分词结果
 - 计算每个分词结果的概率值
 - 返回概率值最高的分词结果

 按照自己的理解，中文分词实际上是求解最优解问题（搜索问题），而n-gram模型在问题中扮演的角色是评估函数，而拥有的状态（即分词结果）的越多，越有可能选出最佳的分词结果。
 所有分词结果最佳获取的方式是穷举，但是穷举通过n-gram计算的结果离散程度太大，效率很低。
 为了能够简化分词结果获取的过程，使用了通过FMM和BMM计算获得两个分词结果。
 这两种算法的分词效果较好，并且可以充分利用词库的优势，筛选掉一些低概率的分词结果，用高效的方式接近最优解。同时也可以让我跟专注于n-gram算法的实现以及后续的实验。


In [120]:
def get_2_gram_p(begin, end, n_grams):
    # 得到分母
    all = sum([count for word, count in n_grams.items() if word.split()[0] == begin])
    # 得到分子
    part = sum([count for word, count in n_grams.items() if word.split()[0] == begin and word.split()[1] == end])
    # 数据平滑（加一法） 计算并返回概率值
    return (part + 1) / (all + len(n_grams))

def get_sentence_p(result, n_grams):
    # 使 i = 1 有意义，添加 <BOS> 和 <Eos>
    result.insert(0, '<BOS>')
    result.append('<EOS>')
    # 计算句子概率值
    p = 1
    for i in range(len(result)-1):
        p *= get_2_gram_p(result[i], result[i+1], n_grams)

    return p

def N_Gram(sentence, words_dic):

    n_grams = get_n_grams()

    bmm_word_list = BMM(sentence, words_dic)
    fmm_word_list = FMM(sentence, words_dic)
    if bmm_word_list == fmm_word_list:
        return bmm_word_list
    else:
        # 计算概率值
        p_bmm = get_sentence_p(bmm_word_list, n_grams)
        p_fmm = get_sentence_p(fmm_word_list, n_grams)
        return bmm_word_list if p_bmm > p_fmm else fmm_word_list
        

# 测试

In [121]:
func_dic = {
    'FMM': FMM,
    'BMM': BMM,
    'Bi-MM': Bi_MM,
    'N-Gram':N_Gram
}

data_name = [
    '1998版新闻',
    'THUOCL清华数据源',
    '词库融合'
]

func = 'N-Gram'
data_name = '词库融合'
sentence = '我们要为中国人民办公益事业' # 为人民办公益事业 我来到北京清华大学
words_dic = get_words_dic(data_name)
result = func_dic[func](sentence, words_dic)
print(result)


['<BOS>', '我们', '要', '为', '中国', '人民', '办公', '益', '事业', '<EOS>']
